In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 500

### Загрузим датасет с машинами. Цель - верно восстанавливать для каждой из них цену продажи!

In [2]:
data = pd.read_csv('autos.csv')

data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,60000,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,135000,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,600000,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,250000,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,450000,141000,Diesel,Individual,Manual,Second Owner


In [3]:
data.info

<bound method DataFrame.info of                                      name  year  selling_price  km_driven  \
0                           Maruti 800 AC  2007          60000      70000   
1                Maruti Wagon R LXI Minor  2007         135000      50000   
2                    Hyundai Verna 1.6 SX  2012         600000     100000   
3                  Datsun RediGO T Option  2017         250000      46000   
4                   Honda Amaze VX i-DTEC  2014         450000     141000   
...                                   ...   ...            ...        ...   
4335  Hyundai i20 Magna 1.4 CRDi (Diesel)  2014         409999      80000   
4336           Hyundai i20 Magna 1.4 CRDi  2014         409999      80000   
4337                  Maruti 800 AC BSIII  2009         110000      83000   
4338     Hyundai Creta 1.6 CRDi SX Option  2016         865000      90000   
4339                     Renault KWID RXT  2016         225000      40000   

        fuel seller_type transmission      

In [3]:
### Колонка с тергетом - "selling price"

X = data.drop("selling_price", axis=1)
y = data["selling_price"]

### Будем замерять MSLE!
### Поэтому прологарифмируем таргет
### А после оптимизируем MSE

y = y.apply(np.log1p)

In [4]:
### Разделим выборку на трейн и тест!

from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

__Задание__ 

Реализуйте свой MeanTargetEncoder с добавленем некоторого шума!

Однажды в лекционном материале, обсуждая счетчики, мы говорили с вами о том, что из-за них модели могут переобучаться. Один из способов бороться с этим - валидировать расчеты среднего таргета (стратегия отложенной выборки / расчеты на кросс-валидации). Но есть еще проще!

Можно просто к значению счетчика добавить случайный шум (зашумить данные)!

Напомним, что рассчитываться новые признаки должны по такой формуле:

$$
g_j = \frac{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)]}{l} + C * \epsilon
$$


Пусть шум будет случайной величиной из нормального стандартного распределения, то есть $\epsilon \sim N(0, 1) $, а $ C = 0.006$.

Создавай свой класс-трансформер, наследуйтесь от классов `BaseEstimator, TransformerMixin` из `sklearn.base`. Трансформер не должен модифицировать передаваемую ему выборку inplace, а все необходимые статистики нужно считать только по обучающей выборке в методе `fit`. 
Ваш трансформер должен принимать при инициализации список из категориальных признаков и список из числовых признаков. 

Если для какого-то признака в тестовой выборке отсутствует значение, трансформер должен поставить там 0.

На выходе должен получиться датасет того же размера с измененными категориальными признаками

In [5]:
object_cols = ['name', 'year', 'fuel', 'seller_type', 'transmission', 'owner']
num_cols = ['km_driven']

X.head()

,name,year,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,141000,Diesel,Individual,Manual,Second Owner


In [9]:
for i in object_cols:
    print(i, pd.unique(X[i]))

name ['Maruti 800 AC' 'Maruti Wagon R LXI Minor' 'Hyundai Verna 1.6 SX' ...
 'Mahindra Verito 1.5 D6 BSIII'
 'Toyota Innova 2.5 VX (Diesel) 8 Seater BS IV'
 'Hyundai i20 Magna 1.4 CRDi']
year [2007 2012 2017 2014 2016 2015 2018 2019 2013 2011 2010 2009 2006 1996
 2005 2008 2004 1998 2003 2002 2020 2000 1999 2001 1995 1997 1992]
fuel ['Petrol' 'Diesel' 'CNG' 'LPG' 'Electric']
seller_type ['Individual' 'Dealer' 'Trustmark Dealer']
transmission ['Manual' 'Automatic']
owner ['First Owner' 'Second Owner' 'Fourth & Above Owner' 'Third Owner'
 'Test Drive Car']


In [30]:
from sklearn.base import BaseEstimator, TransformerMixin

class MeanTargetEncoderNoise(BaseEstimator, TransformerMixin):
    
    def __init__(self, categorical, numeric):
        self.categorical = categorical
        self.numeric = numeric
        
        ### Your code is here
    
    def fit(self, X, y):
        X_fit = X.copy()
        y_fit = y.copy()
        self.target_name = y_fit.name
        X_with_target = pd.concat((X_fit, y_fit), axis=1)
        tresh = np.random.normal(0,1,len(X_fit))*0.006
        ### Your code is here
        self.dict_of_means = {col : X_with_target.groupby(col)[self.target_name].mean()
                              for col in self.categorical}
#         self.dict_of_quantity = {col : X_with_target.groupby(col)[self.target_name].count()
#                                  for col in self.categorical}
        return self
        
    def transform(self, df):
        
        X_ = X.copy()
        len_X = len(X_)
        for col in self.categorical:
                X_[col] = X_[col].map(self.dict_of_means[col])+np.random.normal(0,1,len_X)*0.006
#                 mean_value = transformer.dict_of_means[col].values.mean()
                
#                 X_[col] = X_[col].fillna(mean_value)
        ### Your code is here
        temp = X_
        return temp

In [53]:
from sklearn.base import BaseEstimator, TransformerMixin
import itertools

### Реализуем класс, считающий средние значения по таргету
### И немного зашумляющий их
### Данная реализация не претендует на звание самой лучшей!
### Любые другие, работающие за адекватное время, тоже подойдут
### Советуем потыкать и разобраться, если самостоятельно во время 
### Выполнения ДЗ не получилось!

class MeanTargetEncoderNoise(BaseEstimator, TransformerMixin):
    
    def __init__(self, categorical, numeric):
        self.categorical = categorical
        self.numeric = numeric
    
    def fit(self, X, y):

        X['y'] = y
        
        arr = []
        
        for i in self.categorical:
            
            temp = X.groupby(i).agg({'y':[func1]}).reset_index()
            arr.append((list(temp[i]), list(temp['y']['func1'])))
            
        
        self.arr = arr
        return self.arr
        
    def transform(self, df):
        
        arr = self.arr
        
        temp = pd.DataFrame()
        
        c = 0
        
        for i in self.categorical:
            
            setik = set(df[i].unique())
            setik.difference_update(set(arr[c][0]))

            column = df[i].replace(arr[c][0], arr[c][1]).reset_index()[i]
            column = column.replace(list(setik), 0).reset_index()[i]

            temp = pd.concat([temp, column], axis=1)
            
            c+=1        

        temp = pd.concat([df[self.numeric].reset_index(drop=True), temp], axis=1)
        
        return temp

In [39]:
np.random.seed(1)

### Функция считает среднее и добавляет шум из стандартного нормального распределения

def func1(x):
    return np.sum(x) / x.size + 0.006 * np.random.normal(loc = 0.0, scale = 1.0, size =1)[0]

In [44]:
np.random.seed(1)
transformer = MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)

transformer.fit(X_train, y_train)

train = transformer.transform(X_train)
test = transformer.transform(X_test)

train

,km_driven,name,year,fuel,seller_type,transmission,owner
0,50000,13.483692,13.436559,13.093756,12.615863,13.771135,12.977730
1,70000,12.117029,11.903115,12.453832,12.615863,13.771135,12.977730
2,50000,12.310456,13.328864,12.453832,12.615863,12.639805,12.977730
3,92198,12.491443,13.042359,12.453832,13.152824,12.639805,12.463313
4,3240,12.390906,12.870886,12.453832,12.615863,12.639805,12.463313
...,...,...,...,...,...,...,...
3467,40000,11.853634,11.903115,12.453832,12.615863,12.639805,12.977730
3468,15000,13.135766,13.545966,12.453832,12.615863,12.639805,12.977730
3469,11918,13.303397,13.042359,12.453832,13.646337,12.639805,12.977730
3470,90000,13.909706,13.042359,13.093756,12.615863,12.639805,12.463313


In [54]:
X_train.drop

,name,year,km_driven,fuel,seller_type,transmission,owner,y
3294,Maruti Swift Dzire AMT VDI,2018,50000,Diesel,Individual,Automatic,First Owner,13.422469
2290,Honda City 1.5 V AT,2008,70000,Petrol,Individual,Automatic,First Owner,11.849405
874,Maruti Alto 800 LXI,2017,50000,Petrol,Individual,Manual,First Owner,11.982935
1907,Datsun GO T Petrol,2015,92198,Petrol,Dealer,Manual,Second Owner,12.323860
3244,Hyundai EON Era Plus,2013,3240,Petrol,Individual,Manual,Second Owner,12.542548
...,...,...,...,...,...,...,...,...
2895,Maruti Zen Estilo LXI BSIII,2008,40000,Petrol,Individual,Manual,First Owner,11.561725
2763,Hyundai Grand i10 1.2 Kappa Asta,2019,15000,Petrol,Individual,Manual,First Owner,13.122365
905,Maruti Ertiga VXI,2015,11918,Petrol,Trustmark Dealer,Manual,First Owner,13.345509
3980,Hyundai Creta 1.6 CRDi SX Option,2015,90000,Diesel,Individual,Manual,Second Owner,13.840204


In [43]:
### Проверка работы трансформера

np.random.seed(1)
transformer = MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)

transformer.fit(X_train, y_train)

X_trans = transformer.transform(X_train)

X_trans

,km_driven,name,year,fuel,seller_type,transmission,owner
0,50000,13.483692,13.436559,13.093756,12.615863,13.771135,12.977730
1,70000,12.117029,11.903115,12.453832,12.615863,13.771135,12.977730
2,50000,12.310456,13.328864,12.453832,12.615863,12.639805,12.977730
3,92198,12.491443,13.042359,12.453832,13.152824,12.639805,12.463313
4,3240,12.390906,12.870886,12.453832,12.615863,12.639805,12.463313
...,...,...,...,...,...,...,...
3467,40000,11.853634,11.903115,12.453832,12.615863,12.639805,12.977730
3468,15000,13.135766,13.545966,12.453832,12.615863,12.639805,12.977730
3469,11918,13.303397,13.042359,12.453832,13.646337,12.639805,12.977730
3470,90000,13.909706,13.042359,13.093756,12.615863,12.639805,12.463313


In [35]:
test

,name,year,km_driven,fuel,seller_type,transmission,owner
0,11.376130,11.835237,70000,12.448585,12.614866,12.642519,12.978707
1,11.822716,11.828481,50000,12.446994,12.618933,12.631419,12.966012
2,13.116903,12.628945,100000,13.083166,12.624353,12.630560,12.975083
3,12.374540,13.329905,46000,12.443336,12.621813,12.643670,12.967389
4,13.122913,12.920892,141000,13.083002,12.621235,12.649834,12.464786
...,...,...,...,...,...,...,...
4335,12.852045,12.921031,80000,13.088768,12.614939,12.644477,12.456297
4336,12.925721,12.921735,80000,13.089825,12.621470,12.631076,12.463156
4337,11.370322,12.160205,83000,12.461172,12.617859,12.625488,12.456361
4338,13.909160,13.123795,90000,13.100365,12.618124,12.625356,12.982514


Обучите несколько деревьев, перебирая максимальную глубину алгоритма из списка `max_depth_list`, а остальные параметры оставьте дефолтными. Выведите лучшее значение гиперпараметра. Постройте график зависимости MSLE на тестовой выборке от значения гиперпараметра. Воспользуйтесь `Pipeline` без `GridSearch`. Проделайте то же самое с `min_samples_split`, `min_impurity_decrease`, `max_leaf_nodes`. (по 2б на каждый параметр)

In [45]:
max_depth_list = [3, 5, 8, 12]
min_samples_split_list = [10, 50, 100, 500]
min_impurity_decrease_list = [0, 0.1, 0.15, 0.2]
max_leaf_nodes_list = [100, 200, 500]

In [57]:
X_train = X_train.drop('y', axis = 1)

In [60]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline

np.random.seed(1)

### Your code is here
errors = []
for max_depth in max_depth_list:
    pipe = Pipeline([("custom_transformer",
                  MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),
                  
                  
                 ("decision_tree", 
                  DecisionTreeRegressor(max_depth=max_depth))])
    pipe.fit(X_train, y_train)
    
    preds = pipe.predict(X_test)

    errors.append(mse(preds, y_test, squared=True))





AttributeError: 'list' object has no attribute 'transform'

Подберите лучшую комбинацию параметров, используя `GridSearchCV` и набор массивов значений параметров из предыдущего задания. Для лучшей комбинации посчитайте MSLE на тестовой выборке. Получились ли лучшие параметры такими же, как если бы вы подбирали их по-отдельности при остальных гиперпараметрах по умолчанию (предыдущее задание)? (2б)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "decision_tree__max_depth": [3, 5, 8, 12],
    "decision_tree__min_samples_split": [10, 50, 100, 500],
    "decision_tree__min_impurity_decrease": [0, 0.1, 0.15, 0.2],
    "decision_tree__max_leaf_nodes": [100, 200, 500]
}
np.random.seed(1)

### Your code is here



